In [149]:
# Data processing
import numpy as np
import pandas as pd

# Download data
import yfinance as yf

from scipy.stats import norm

In [140]:
# Initial investment and currency per stock

# in dollar values initially?

initial_investment = 100000000 #100mil

# initial weights
weights = np.array([0.2,0.2,0.2,0.2,0.2])

investment = {
    "DBA" : [20000000, 'USD'], # iPath Pure Beta Broad Commodity ETN (BCM)
    "^HSI" : [20000000, 'USD'], # Hang Seng Index
    "^AEX"  : [20000000, 'EUR'], # AEX Index
    "^GSPC" : [20000000, 'USD'], # S&P 500 Index
    "^N225" : [20000000, 'JPY'] # Nikkei 225
}

start = "2010-01-01"
end = "2018-12-31"

In [127]:
investment.values()

dict_values([[1000000, 'USD'], [2500000, 'USD'], [500000, 'EUR'], [2300000, 'USD'], [1750000, 'JPY']])

In [83]:
# Download data and store in dataframe

instruments_data = yf.download(' '.join(list(investment.keys())), start=start, end=end, groupby='ticker')
instruments = instruments_data['Adj Close']

[*********************100%***********************]  5 of 5 completed


In [84]:
instruments.tail()

,DBA,^AEX,^GSPC,^HSI,^N225
Date,,,,,
2018-12-24,16.687899,478.339996,2351.100098,25651.380859,NaN
2018-12-25,NaN,NaN,NaN,NaN,19155.740234
2018-12-26,16.697742,NaN,2467.699951,NaN,19327.060547
2018-12-27,16.638670,476.029999,2488.830078,25478.880859,20077.619141
2018-12-28,16.707588,484.170013,2485.739990,25504.199219,20014.769531


In [85]:
# Create exchange rate ticker list based on initial investment data
currencies = list(set(['EUR' + i[1] + '=X' for i in investment.values()]))

# Remove EUR-EUR exchange rate
for currency in currencies:
    if currency[0:2] == currency[3:5]:
        currencies.remove(currency)
        
currencies = " ".join(currencies)

In [86]:
currencies

'EURUSD=X EURJPY=X'

In [87]:
# Download exchange rate data
exchangerate_data = yf.download(currencies, start=start, end=end, groupby='ticker')
exchangerate = exchangerate_data['Adj Close']
exchangerate.tail()

[*********************100%***********************]  2 of 2 completed


,EURJPY=X,EURUSD=X
Date,,
2018-12-24,126.292000,1.137281
2018-12-25,125.769997,1.140394
2018-12-26,126.001999,1.141553
2018-12-27,126.404999,1.136131
2018-12-28,126.767998,1.143105


In [88]:
# Scrape EURIBOR rates

import csv
import requests
import io

year_range = int(end[0:4]) - int(start[0:4]) + 1
years = range(int(start[0:4]), int(start[0:4]) + year_range)

urls = ['https://www.emmi-benchmarks.eu/assets/components/rateisblue/file_processing/publication/processed/hist_EURIBOR_{}.csv'
        .format(i) for i in range(int(start[0:4]), int(start[0:4]) + year_range)]

for url in urls:
    s = requests.get(url).content
    if url == urls[0]:
        euribor = pd.read_csv(io.StringIO(s.decode('utf-8'))).transpose()
    else:
        euribor = euribor.append(pd.read_csv(io.StringIO(s.decode('utf-8'))).transpose())
        
error_lines = ['Unnamed: 0', 'Unnamed: 261']

euribor3m = euribor[~euribor.index.isin(error_lines)][5]

In [89]:
# Make index datetime
euribor3m.index = pd.to_datetime(euribor3m.index, format='%d/%m/%Y')

In [90]:
# holding 10% in cash

euribor3m

2010-01-04      0.7
2010-01-05    0.699
2010-01-06    0.694
2010-01-07    0.692
2010-01-08    0.691
              ...  
2018-12-21   -0.238
2018-12-24   -0.237
2018-12-27   -0.237
2018-12-28   -0.236
2018-12-31   -0.237
Name: 5, Length: 2306, dtype: object

In [91]:
# invest in stock (and commodity) indices (by investing in mutual funds)? ETFs?


instruments

,DBA,^AEX,^GSPC,^HSI,^N225
Date,,,,,
2009-12-31,25.755527,NaN,1115.099976,NaN,NaN
2010-01-04,25.999054,343.029999,1132.989990,21823.279297,10654.790039
2010-01-05,26.164654,342.019989,1136.520020,22279.580078,10681.830078
2010-01-06,26.310770,341.480011,1137.140015,22416.669922,10731.450195
2010-01-07,26.106207,340.549988,1141.689941,22269.449219,10681.660156
...,...,...,...,...,...
2018-12-24,16.687899,478.339996,2351.100098,25651.380859,NaN
2018-12-25,NaN,NaN,NaN,NaN,19155.740234
2018-12-26,16.697742,NaN,2467.699951,NaN,19327.060547


In [92]:
# for exchange rate conversion

exchangerate

,EURJPY=X,EURUSD=X
Date,,
2009-12-31,133.149994,1.432706
2010-01-01,133.710007,1.438994
2010-01-04,133.389999,1.442398
2010-01-05,131.529999,1.436596
2010-01-06,133.013000,1.440403
...,...,...
2018-12-24,126.292000,1.137281
2018-12-25,125.769997,1.140394
2018-12-26,126.001999,1.141553


### Step2: 
Implement and backtest VaR and ES for the two abovementioned confidence levels and 1-day horizon using the following methods:

In [93]:
from functools import reduce

In [116]:
# merge the instrument prices, exchange rates, and euribor into one dataframe
dfs = [instruments, exchangerate, euribor3m]
df_pf = reduce(lambda left,right: pd.merge(left,right,left_index=True, right_index=True), dfs)
df_pf = df_pf.rename(columns={5: 'euribor3m', '^AEX':'AEX', '^GSPC':'GSPC','^N225':'N225', '^HSI':'HSI'})
df_pf['euribor3m'] = pd.to_numeric(df_pf['euribor3m'])

In [111]:
instrument_names = df_pf.columns[:5]
instrument_names

Index(['DBA', 'AEX', 'GSPC', 'HSI', 'N225'], dtype='object')

In [102]:
# Estimate VaR and ES, based on Variance-Covariance method, multivariate normal distribution

In [120]:
df_returns = np.log(df_pf[instrument_names])- np.log(df_pf[instrument_names].shift())

In [122]:
df_returns

,DBA,AEX,GSPC,HSI,N225
2010-01-04,NaN,NaN,NaN,NaN,NaN
2010-01-05,0.006349,-0.002949,0.003111,0.020693,0.002535
2010-01-06,0.005569,-0.001580,0.000545,0.006134,0.004635
2010-01-07,-0.007805,-0.002727,0.003993,-0.006589,-0.004650
2010-01-08,0.002981,0.004073,0.002878,0.001225,0.010862
...,...,...,...,...,...
2018-12-20,-0.002913,-0.020409,-0.015898,-0.009395,-0.028776
2018-12-21,-0.008201,-0.002390,-0.020803,0.005056,-0.011164
2018-12-24,0.007699,-0.013435,-0.027487,-0.003970,NaN
2018-12-27,-0.002954,-0.004841,0.056929,-0.006747,NaN


In [123]:
# calculate the variance-covariance matrix
cov_matrix = df_returns.cov()
cov_matrix

,DBA,AEX,GSPC,HSI,N225
DBA,0.000066,0.000021,0.000023,0.000012,0.000012
AEX,0.000021,0.000114,0.000064,0.000048,0.000047
GSPC,0.000023,0.000064,0.000091,0.000022,0.000022
HSI,0.000012,0.000048,0.000022,0.000129,0.000081
N225,0.000012,0.000047,0.000022,0.000081,0.000178


In [124]:
# calculate the mean return
avg_returns = df_returns.mean()

In [147]:
# calcuclate the portfolio mean
pf_mean_pct = sum(avg_returns*weights)
pf_mean_pct

0.00012677748676323699

In [145]:
# calculate the portfolio standard deviation in percentage terms

pf_std_pct = np.sqrt(weights.T.dot(cov_matrix).dot(weights))

In [148]:
# convert the pf return and std from percentage to euro
pf_mean_euro = pf_mean_pct * initial_investment
pf_std_euro = pf_std_pct * initial_investment

In [151]:
confidence_levels = [0.975, 0.99]
alphas = [0.025,0.01]
alphas

[0.025, 0.01]

In [162]:
# calculate the VaR
var_975_1d = norm.ppf(confidence_levels[0]) * pf_std_euro - pf_mean_euro
es_975_1d = alphas[0]**-1 * norm.pdf(norm.ppf(alphas[0]))*pf_std_euro - pf_mean_euro

In [159]:
# var_990_1d = norm.ppf(confidence_levels[1], pf_mean_euro, pf_std_euro)
# var_990_1d

In [161]:
var_990_1d = norm.ppf(confidence_levels[1]) * pf_std_euro - pf_mean_euro
es_990_1d = alphas[1]**-1 * norm.pdf(norm.ppf(alphas[1]))*pf_std_euro - pf_mean_euro

In [163]:
print(f'VaR 99% 1d: {var_990_1d}, ES 99% 1d: {es_990_1d}')

VaR 99% 1d: 1651657.5870184936, ES 99% 1d: 1894092.1947255149
